# Import catalog from postgres

In [41]:
import psycopg2
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, box
import datetime

# Import catalog from Postgress
conn = psycopg2.connect(database="mydb",
  user="user",
  host="localhost",
  password="user")
catalog = gpd.read_postgis('select * from metadata_catalog', conn, geom_col='bbox')
catalog.head()

,id,layer_name,layer_type,n_layers,publication_date,reference_date,covering_period,layer_region,lang,abstract,keywords,wms_preview,grs,bbox,contains_timeseries,origin,contact_details
0,1,DNI_yearly,raster,1,2020-04-07,2018-12-31,"[1994-01-01, 2019-01-01)",Greece,ENG,Longterm yearly average of direct normal irrad...,"Direct normal irradiance, DNI, raster, WCS, WMS",http://localhost:8082/geoserver/geonode/wms?se...,WGS84,"POLYGON ((19.374 34.802, 19.374 41.749, 29.643...",False,https://solargis.com/,"Sotiris Pelekis, email: pelekhs@gmail.com"
1,2,NDVI_20010101,raster,1,2020-04-07,2001-01-01,"[2001-01-01, 2001-01-02)",World,ENG,Single channel NDVI for the whole world for 20...,"NDVI, raster, WCS, WMS",http://localhost:8082/geoserver/geonode/wms?se...,WGS84,"POLYGON ((-180.000 -90.000, -180.000 83.634, 1...",False,https://www.usgs.gov/,"Sotiris Pelekis, email: pelekhs@gmail.com"
2,3,NDVI_20050101,raster,1,2020-04-07,2005-01-01,"[2005-01-01, 2005-01-02)",World,ENG,Single-channel NDVI for the whole world for 20...,"NDVI, raster, WCS, WMS",http://localhost:8082/geoserver/geonode/wms?se...,WGS84,"POLYGON ((-180.000 -90.000, -180.000 83.634, 1...",False,https://www.usgs.gov/,"Sotiris Pelekis, email: pelekhs@gmail.com"
3,4,NDVI_20050301,raster,1,2020-04-07,2005-03-01,"[2005-03-01, 2005-03-02)",World,ENG,Single-channel NDVI for the whole world for 20...,"NDVI, raster, WCS, WMS",http://localhost:8082/geoserver/geonode/wms?se...,WGS84,"POLYGON ((-180.000 -90.000, -180.000 83.634, 1...",False,https://www.usgs.gov/,"Sotiris Pelekis, email: pelekhs@gmail.com"
4,5,NDVI_20050601,raster,1,2020-04-07,2005-06-01,"[2005-06-01, 2005-06-02)",World,ENG,Single-channel NDVI for the whole world for 20...,"NDVI, raster, WCS, WMS",http://localhost:8082/geoserver/geonode/wms?se...,WGS84,"POLYGON ((-180.000 -90.000, -180.000 83.634, 1...",False,https://www.usgs.gov/,"Sotiris Pelekis, email: pelekhs@gmail.com"


# Interactive queries on catalog

In this section I create an interactive search and filtering application that performs queries on the catalog based on the selections of the user in the provided GUI. The application is based on the ipywidgets library. It perimits the user to perform various queries on the catalog.

## Helping functions

This cell consists of two functions-tools for the application.

In [2]:
date_query = ("""SELECT * FROM metadata_catalog WHERE '[{}, {}]'::daterange <@ covering_period AND covering_period <> 'EMPTY'""")
# list comparison assistant function
def common_elements(list1, list2):
    return [element for element in list1 if element in list2]

# Date filtering assistant function
def date_range_in_covering_period(d1, d2):
    if d1 and d2 and d1.year in range(1000, 2100) and d2.year in range(1000,2100) and d1.month in range(1,13) and d2.month in range(1,13):
        try:
            d1s=d1.strftime("%Y-%m-%d")
        except AttributeError:
            d1s="No date chosen"
        try:
            d2s=d2.strftime("%Y-%m-%d")
        except AttributeError:
            d2s="No date chosen"
        #display("Start date:", d1s, "End date:", d2s)
        if (d1s!="No date chosen") and (d2s!="No date chosen") and d1<=d2:
            conn = psycopg2.connect(database="mydb",
                                    user="user",
                                    host="localhost",
                                    password="user")
            result = list(gpd.read_postgis(date_query.format(d1,d2),
                                      conn, geom_col='bbox').layer_name.values)
            return result
        elif d1>d2:
            return("Wrong dates")
        else:
            return("No dates chosen")
    else:
        return ("Wrong dates")

This cell constitutes the **core of the application** and its result is a **Query Building UI**.

In [25]:
import datetime
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive

## Widgets initialisation

# Date range widget
calendar1 = widgets.DatePicker(description='From', disabled=False)
calendar2 = d2=widgets.DatePicker(description='To', disabled=False)

# Timeseries widget
ts_checkbox = widgets.Checkbox(value=False, description='Timeseries only')

#BBOX coordinates widgets
x1=widgets.FloatSlider(min=-180, max=180, step=0.0001, value=23, readout_format='.4f', name="Xmin")
x2= widgets.FloatSlider(min=-180, max=180, step=0.0001, value=24, readout_format='.4f') 
y1=widgets.FloatSlider(min=-90, max=90, step=0.0001, value=37, readout_format='.4f')
y2= widgets.FloatSlider(min=-90, max=90, step=0.0001, value=38, readout_format='.4f')

#Dropdown widgets
type_dropdown = widgets.Dropdown(
    options=["Any"]+list(catalog.layer_type.unique()),
    description='Layer Type:',
    disabled=False,
)
region_dropdown = widgets.Dropdown(
    options=["Any"]+list(catalog.layer_region.unique()),
    description='Region:',
    disabled=False,
)

#Output Variable so as to keep output refreshed
output = widgets.Output()
date_query = ("""SELECT * FROM metadata_catalog WHERE '[{}, {}]'::daterange <@ covering_period AND covering_period <> 'EMPTY'""")

# Filter ensembling function
def common_filtering(date1, date2, ts, x1, y1, x2, y2, layer_type, region):
    #clear output at every refresh
    output.clear_output()
    
    #init common filter
    common_filter=list(catalog.layer_name)
    
    # Timeseries filtering
    if ts:
        ts_filter = list(catalog[catalog["contains_timeseries"]==True].layer_name)
        common_filter=common_elements(common_filter, ts_filter)
    
    # Bbox filtering
    bbox_filter = list(catalog[[catalog.loc[i]["bbox"].contains(box(x1, y1, x2, y2)) 
                           for i in range(len(catalog))]].layer_name.values)
    common_filter = common_elements(common_filter, bbox_filter)
    
    # Layer type filtering
    if layer_type != "Any":
        layer_filter = list(catalog[catalog["layer_type"]==layer_type].layer_name)
        common_filter = common_elements(common_filter, layer_filter)
        
    # Region filtering
    if region != "Any":
        region_filter = list(catalog[catalog["layer_region"]==region].layer_name)
        common_filter = common_elements(common_filter, region_filter)
    
    #Date filtering
    date_filter = date_range_in_covering_period(date1, date2)
    if isinstance(date_filter, list):
        common_filter = common_elements(common_filter, date_filter)
    #elif date_filter=="Wrong dates":
     #   with output:
      #      display("Dates are wrong and are not taken into consideration in this search")
    with output:
        display(common_filter)
        return common_filter

# Handler Functions for event handling on the GUI

def ts_handler(change):
    common_filtering(calendar2.value, calendar1.value, change.new, x1.value, y1.value,
                     x2.value, y2.value, type_dropdown.value, region_dropdown.value)

def date1_handler(change):
    common_filtering(change.new, calendar2.value, ts_checkbox.value, x1.value, y1.value,
                     x2.value, y2.value, type_dropdown.value, region_dropdown.value)

def date2_handler(change):
    common_filtering(calendar1.value, change.new, ts_checkbox.value, x1.value, y1.value,
                     x2.value, y2.value, type_dropdown.value, region_dropdown.value)

def x1_handler(change):
    common_filtering(calendar1.value, calendar2.value, ts_checkbox.value, change.new, y1.value,
                     x2.value, y2.value, type_dropdown.value, region_dropdown.value)

def y1_handler(change):
    common_filtering(calendar1.value, calendar2.value, ts_checkbox.value, x1.value, change.new,
                     x2.value, y2.value, type_dropdown.value, region_dropdown.value)

def x2_handler(change):
    common_filtering(calendar1.value, calendar2.value, ts_checkbox.value, x1.value, y1.value,
                     change.new, y2.value, type_dropdown.value, region_dropdown.value)

def y2_handler(change):
    common_filtering(calendar1.value, calendar2.value, ts_checkbox.value, x1.value, y1.value,
                     x2.value, change.new, type_dropdown.value, region_dropdown.value)
    
def type_dropdown_handler(change):
    common_filtering(calendar1.value, calendar2.value, ts_checkbox.value, x1.value, y1.value,
                     x2.value, y2.value, change.new, region_dropdown.value)
def region_dropdown_handler(change):
    common_filtering(calendar1.value, calendar2.value, ts_checkbox.value, x1.value, y1.value,
                     x2.value, y2.value, type_dropdown.value, change.new)
    
# Event observation

calendar1.observe(date1_handler, names="value")
calendar2.observe(date2_handler, names="value")
ts_checkbox.observe(ts_handler, names="value")
x1.observe(x1_handler, names="value")
y1.observe(x2_handler, names="value")
x2.observe(y1_handler, names="value")
y2.observe(y2_handler, names="value")
type_dropdown.observe(type_dropdown_handler, names="value")
region_dropdown.observe(region_dropdown_handler, names="value")

#Display widgets

print("Check if you want the layer to contain timeseries:")
display(ts_checkbox)
print("Choose bounding box to be included in the layer:")
display("Xmin")
display(x1)
display("Ymin")
display(y1)
display("Xmax")
display(x2)
display("Ymax")
display(y2)
display(type_dropdown)
display(region_dropdown)
print("Choose date range to be included in the layer:")
display(calendar1)
display(calendar2)

Check if you want the layer to contain timeseries:


Checkbox(value=False, description='Timeseries only')

Choose bounding box to be included in the layer:


'Xmin'

FloatSlider(value=23.0, max=180.0, min=-180.0, readout_format='.4f', step=0.0001)

...

Dropdown(description='Layer Type:', options=('Any', 'raster', 'vector'), value='Any')

Dropdown(description='Region:', options=('Any', 'Greece', 'World'), value='Any')

Choose date range to be included in the layer:


DatePicker(value=None, description='From')

DatePicker(value=None, description='To')

Below we get a live update of the above query results:

In [36]:
display(output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '[]'}, 'metadata': {}},))

In the next section we preview the above results via geoserver and we offer downloading options to the user:

# Preview & Download options through WMS, WCS & WFS (using OWSlib)

In this section we use OWSlib in order to offer download options for the above selected layers. The user can see a preview of the layers that match to the above applied filters and click download in order to download the image.

Below I use the functions built and tested inside the file: WMS_WFS_WCS_experimentation.ipynb

### WMS connection and download service

In [37]:
from owslib.wms import WebMapService
import os, sys
url = 'http://localhost:8082/geoserver/ows'
wms = WebMapService(url, version='1.1.1')

def getMap(layer, bbox=None, workspace="geonode", style=None):
    if bbox:
        response = wms.getmap(layers=[layer],
                         size=(688,398),
                         srs='EPSG:4326',
                         bbox=bbox,
                         format= 'image/png', 
                         transparent=True,
                         style=style)
    else:
        response = wms.getmap(layers=[layer],
                 size=(688,398),
                 srs='EPSG:4326',
                 bbox=wms[workspace+":"+layer].boundingBoxWGS84,
                 format= 'image/png', 
                 transparent=True,
                 style=style)
    webbrowser.open(response.geturl())
    f = open (os.path.join("wms",layer), 'wb')
    f.write(response.read())
    f.close()
    return response.geturl()

### WFS connection and download service

In [38]:
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url=url, version='1.1.0')
wfs.identification.title

def getFeature(layer, bbox=None, workspace="geonode", format="json"):
    if bbox:
        response = wfs.getfeature(typename=workspace+":"+layer, outputFormat=format, bbox=bbox)
    else:
        response = wfs.getfeature(typename=workspace+":"+layer, outputFormat=format,
                                  bbox=wfs[workspace+":"+layer].boundingBoxWGS84)
    with open(os.path.join("wfs", layer+"."+format), 'wb') as f:
        while True:
            data = response.read(100)
            if not data:
                break
            f.write(data)
    name=f.name
    f.close
    return (name)

### WCS connection and download service

In [39]:
from owslib.wcs import WebCoverageService
import webbrowser

wcs = WebCoverageService(url=url, version='1.0.0')

def wcs_download(layer, bbox=None, workspace="geonode", format="GeoTIFF", height=512, width=512):
    if bbox:
        response = wcs.getCoverage(identifier="geonode"+":"+layer, format=format, height=height, width=width, crs="EPSG:4326", bbox=bbox)
    else:
        response = wcs.getCoverage(identifier="geonode"+":"+layer, format=format, height=height, width=width, crs="EPSG:4326")
    webbrowser.open(response.geturl())
    return response.geturl()

# Preview and Interactive Download application

In [40]:
import os
from IPython.display import Image, IFrame, display

# Initialisations
#query results from above
query_results = output.outputs[0]["data"]["text/plain"].\
replace("'","").replace("[","").replace("]","").replace("\n","").\
replace(" ","").split(",") if len(output.outputs)>0 else list(catalog.layer_name)

# Widgets

dropdown = widgets.Dropdown(options=query_results)

button = widgets.Button(
    value=False,
    description='Download options',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='info' # (FontAwesome names without the `fa-` prefix)
)
WMS_button = widgets.Button(
    value=False,
    description='Image',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
WFS_button = widgets.Button(
    value=False,
    description='Vector',
    disabled=True,
    button_style='',
    tooltip='Description',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
WCS_button = widgets.Button(
    value=False,
    description='Raster',
    disabled=True,
    button_style='',
    tooltip='Description',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
# bbox checkbox
bbox = widgets.Checkbox(value=False, description='Use bounding box from search')

# output
output = widgets.Output()

#Preview function
def show_geoserver_preview(layer_name = dropdown):
    print('Please choose the layer from the menu that you want to download')
    display(IFrame(src=catalog[catalog["layer_name"]==layer_name]["wms_preview"].values[0] ,width=700, height=600))
    return layer_name

# Download options: Enable WCS only for raster and WFS only for vector layers
def download_options(layer_name):
    with output:
        #all
        display(WMS_button)
        WMS_button.on_click(on_WMS_button_clicked) 
        #raster
        WCS_button.disabled = True
        if catalog[catalog["layer_name"]==layer_name]["layer_type"].values[0]=="raster":
            WCS_button.disabled = False
        display(WCS_button)
        WCS_button.on_click(on_WCS_button_clicked)
        #vector
        WFS_button.disabled = True
        if catalog[catalog["layer_name"]==layer_name]["layer_type"].values[0]=="vector":
            WFS_button.disabled = False
        display(WFS_button)
        WFS_button.on_click(on_WFS_button_clicked)

# event handlers
def on_WMS_button_clicked(b):
    #with output:
    print("The file " + dropdown.value + " is being downloaded using WMS as PNG")
    if bbox.value:
        return getMap(dropdown.value, [x1.value, y1.value, x2.value, y2.value])
    else:
        return getMap(dropdown.value)
def on_WCS_button_clicked(b):
    #with output:
    print("The file " + dropdown.value + " is being downloaded using WCS as GeoTiff")
    if bbox.value:
        return wcs_download(dropdown.value, [x1.value, y1.value, x2.value, y2.value])
    else:
        return wcs_download(dropdown.value)
    
def on_WFS_button_clicked(b):
    #with output:
    print("The file " + dropdown.value + " is being downloaded using WFS as GeoJson")
    if bbox.value:
        return getFeature(dropdown.value, [x1.value, y1.value, x2.value, y2.value])
    else:
        return getFeature(dropdown.value)
def on_button_clicked(b):
    output.clear_output()
    with output:
        print("Choose if you need data or image")
    return download_options(dropdown.value)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        output.clear_output()

# preview
preview = interactive(show_geoserver_preview)
display(preview)
dropdown.observe(on_change)

# bbox
display(bbox)

# Downloads
display(button)
button.on_click(on_button_clicked)


display(output)

interactive(children=(Dropdown(description='layer_name', options=('DNI_yearly', 'PVOUT', 'PVOUT_yearly', 'TEMP…

Checkbox(value=False, description='Use bounding box from search')

Button(description='Download options', icon='info', style=ButtonStyle(), tooltip='Description')

Output()